**1. Upload the video file to data folder**


In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Відкриваємо файли
cap = cv2.VideoCapture("data/Match.mp4")
template = cv2.imread("data/template.png", 0)
h, w = template.shape[:2]

In [3]:
# Output data
output_size = (1080, 1920)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("data/output2.mp4", fourcc, 30.0, output_size)

In [4]:
# Source points from video frame
pts_src = np.array([[210, 60], [945, 60], [1113, 1240], [114, 1240]], dtype=np.float32)

# Corresponding points on the "straightened" image
pts_dst = np.array([[0, 0], [1080, 0], [1080, 1920], [0, 1920]], dtype=np.float32)

# Homography matrix for perspective correction
H, _ = cv2.findHomography(pts_src, pts_dst)

In [5]:
tracker = None
detect_threshold = 0.83  # For template matching
template_verify_threshold = 0.65  # For checking if tracking is still on the ball
frame_count = 0

In [6]:
last_valid_positions = []  # Store last few valid positions to detect jumps
max_positions = 15  # Number of positions to keep in history
max_jump_distance = 50  # Maximum allowed jump distance between frames

⚠️ **To Fix:** Трекінг в цілому працює, але коли гравець різко б'є по м'ячу, то трекінг переключається на гравця і застрягає на ньому


In [7]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply perspective transformation to the frame
    warped = cv2.warpPerspective(frame, H, (1080, 1920))
    gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)  # у сірий

    # Search for the ball
    if tracker is None:
        # If tracking is not active - search for the ball using template matching
        res = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        print(f"[Detection] Accuracy: {max_val:.3f}")

        # If the match value exceeds the threshold - object found
        if max_val >= detect_threshold:
            print(f"[Detection] Match: {max_val:.3f}")
            top_left = max_loc
            bbox = (top_left[0], top_left[1], w, h)

            # Initialize tracker
            tracker = cv2.legacy.TrackerCSRT_create()
            tracker.init(warped, bbox)

            bottom_right = (top_left[0] + w, top_left[1] + h)

            cv2.rectangle(warped, top_left, bottom_right, (255, 0, 0), 2)
            cv2.putText(
                warped,
                f"Detected {max_val:.3f}",
                (top_left[0], top_left[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (255, 0, 0),
                2,
            )

            # Add position to history
            ball_center = (top_left[0] + w // 2, top_left[1] + h // 2)
            last_valid_positions.append(ball_center)
            if len(last_valid_positions) > max_positions:
                last_valid_positions.pop(0)
    else:
        frame_count += 1
        success, bbox = tracker.update(warped)
        print(f"[Tracking] Tracker active, bbox: {bbox}")

        if success:
            x, y, bw, bh = [int(v) for v in bbox]
            current_center = (x + bw // 2, y + bh // 2)

            # Check if ball jumped too far - this indicates tracker drift
            if len(last_valid_positions) > 0:
                last_pos = last_valid_positions[-1]
                distance = np.sqrt(
                    (current_center[0] - last_pos[0]) ** 2
                    + (current_center[1] - last_pos[1]) ** 2
                )
                if distance > max_jump_distance:
                    print(
                        f"[Warning] Ball jumped too far ({distance:.1f} pixels), likely tracker drift"
                    )
                    tracker = None
                    continue

            # 1. Template verification check
            if frame_count % 20 == 0:  # Check template every 20 frames
                roi = gray[y : y + bh, x : x + bw]
                if roi.shape[0] >= h and roi.shape[1] >= w:
                    roi_resized = cv2.resize(roi, (w, h))
                    res = cv2.matchTemplate(roi_resized, template, cv2.TM_CCOEFF_NORMED)
                    _, match_val, _, _ = cv2.minMaxLoc(res)
                    print(f"[Tracking] Match to template: {match_val:.3f}")

                    if match_val < template_verify_threshold:
                        print("[Warning] Tracker drifted from template - resetting")
                        tracker = None
                        continue

            # 2. Circularity check
            roi = gray[
                max(0, y) : min(gray.shape[0], y + bh),
                max(0, x) : min(gray.shape[1], x + bw),
            ]
            if roi.size > 0:  # Make sure ROI is valid
                # Apply thresholding to isolate the ball from background
                _, roi_thresh = cv2.threshold(roi, 170, 255, cv2.THRESH_BINARY)

                # Apply morphological operations to clean up the mask
                kernel = np.ones((3, 3), np.uint8)
                roi_thresh = cv2.morphologyEx(roi_thresh, cv2.MORPH_OPEN, kernel)
                roi_thresh = cv2.morphologyEx(roi_thresh, cv2.MORPH_CLOSE, kernel)

                contours, _ = cv2.findContours(
                    roi_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
                )
                valid_contour = False

                for cnt in contours:
                    if len(cnt) > 5:  # Need at least 5 points to fit an ellipse
                        area = cv2.contourArea(cnt)
                        if area > 30:  # Ignore tiny contours
                            perimeter = cv2.arcLength(cnt, True)
                            circularity = 4 * np.pi * area / (perimeter**2 + 1e-5)
                            print(f"[Circularity] Value: {circularity:.3f}")

                            if circularity > 0.7:  # Closer to 1 = circle
                                valid_contour = True
                                # Draw the circular contour
                                cv2.drawContours(
                                    warped,
                                    [cnt + np.array([max(0, x), max(0, y)])],
                                    -1,
                                    (0, 255, 0),
                                    2,
                                )

                if (
                    not valid_contour and frame_count % 5 == 0
                ):  # Only check every 5 frames to avoid flickering
                    print("[Warning] No circular object in ROI - might have drifted")
                    # We'll let template matching confirm this in next verification

            # If we've passed all checks, update history
            last_valid_positions.append(current_center)
            if len(last_valid_positions) > max_positions:
                last_valid_positions.pop(0)

            # Draw rectangle around the tracked ball
            cv2.rectangle(warped, (x, y), (x + bw, y + bh), (255, 0, 0), 2)
            cv2.putText(
                warped,
                f"Tracked",
                (x + 10, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (255, 0, 0),
                2,
            )

            # Draw trajectory
            if len(last_valid_positions) > 1:
                for i in range(1, len(last_valid_positions)):
                    cv2.line(
                        warped,
                        last_valid_positions[i - 1],
                        last_valid_positions[i],
                        (0, 0, 255),
                        2,
                    )
        else:
            print("[Info] Tracker lost - retrying.")
            tracker = None

    # Записуємо оброблений кадр у відео
    out.write(warped)

    # зменшую розмір кадру в 2 рази для зручного перегляду
    scaled_warped = cv2.resize(warped, (0, 0), fx=0.5, fy=0.5)
    cv2.imshow("Warped View", scaled_warped)

    # вихід по клавіші Esc
    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

[Detection] Accuracy: 0.768
[Detection] Accuracy: 0.770
[Detection] Accuracy: 0.774
[Detection] Accuracy: 0.782
[Detection] Accuracy: 0.761
[Detection] Accuracy: 0.763
[Detection] Accuracy: 0.771
[Detection] Accuracy: 0.769
[Detection] Accuracy: 0.780
[Detection] Accuracy: 0.773
[Detection] Accuracy: 0.790
[Detection] Accuracy: 0.772
[Detection] Accuracy: 0.783
[Detection] Accuracy: 0.791
[Detection] Accuracy: 0.785
[Detection] Accuracy: 0.777
[Detection] Accuracy: 0.773
[Detection] Accuracy: 0.782
[Detection] Accuracy: 0.807
[Detection] Accuracy: 0.796
[Detection] Accuracy: 0.784
[Detection] Accuracy: 0.774
[Detection] Accuracy: 0.776
[Detection] Accuracy: 0.777
[Detection] Accuracy: 0.775
[Detection] Accuracy: 0.772
[Detection] Accuracy: 0.776
[Detection] Accuracy: 0.758
[Detection] Accuracy: 0.774
[Detection] Accuracy: 0.783
[Detection] Accuracy: 0.777
[Detection] Accuracy: 0.783
[Detection] Accuracy: 0.780
[Detection] Accuracy: 0.770
[Detection] Accuracy: 0.798
[Detection] Accuracy